## Импорты

In [1]:
import random
from collections import deque, Counter
import numpy as np
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest

## Константы

In [2]:
# Типы карт
CARD_TYPES = [
    'crab', 'ship', 'fish', 'shark', 'swimmer',
    'shell', 'octopus', 'penguin', 'sailor'
]

# Стартовое количество карт
STARTING_COUNTS = {
    'crab': 9,
    'ship': 8,
    'fish': 7,
    'shark': 5,
    'swimmer': 5,
    'shell': 6,
    'octopus': 5,
    'penguin': 3,
    'sailor': 2,
}

# Парные и непарные карты
PAIRABLE = {'crab', 'ship', 'fish', 'shark', 'swimmer'}
UNPAIRED = {'shell', 'octopus', 'penguin', 'sailor'}

# Таблица очков
SHELL_SCORES = {1: 0, 2: 2, 3: 4, 4: 6, 5: 8, 6: 10}
OCTOPUS_SCORES = {1: 0, 2: 3, 3: 6, 4: 9, 5: 12}
PENGUIN_SCORES = {1: 1, 2: 3, 3: 5}
SAILOR_SCORES = {1: 0, 2: 5}

## Генерация базовой стратегии

In [3]:
def generate_strategy():
    top2 = ['penguin', 'ship']
    random.shuffle(top2)
    rest = [c for c in CARD_TYPES if c not in top2]
    random.shuffle(rest)
    return top2 + rest

## Генерация альтернативных стратегий

In [4]:
def reverse_strategy(strat):
    return strat[::-1]

def swap_paired_strategy(strat):
    new = strat.copy()

    paired_top2 = [c for c in new[:2] if c in PAIRABLE]
    paired = paired_top2[0]
    idx_p = new.index(paired)

    indices = [i for i, c in enumerate(new[2:], start=2) if c in PAIRABLE]
    i = random.choice(indices)

    new[idx_p], new[i] = new[i], new[idx_p]
    return new


def swap_unpaired_strategy(strat):
    new = strat.copy()

    unpaired_top2 = [c for c in new[:2] if c in UNPAIRED]
    up = unpaired_top2[0]
    idx_u = new.index(up)

    indices = [i for i,c in enumerate(new[2:], start=2) if c in UNPAIRED]
    i = random.choice(indices)

    new[idx_u], new[i] = new[i], new[idx_u]
    return new

## Подсчет очков в руке

In [5]:
def score_sets(hand):
    cnt = Counter(hand)
    score = 0
    score += SHELL_SCORES.get(cnt.get('shell', 0), 0)
    score += OCTOPUS_SCORES.get(cnt.get('octopus', 0), 0)
    score += PENGUIN_SCORES.get(cnt.get('penguin', 0), 0)
    score += SAILOR_SCORES.get(cnt.get('sailor', 0), 0)
    return score

## Симуляция раунда

### Вспомогательные функции для взятия карт

In [6]:
def get_from_discards(strat, discards):
    card = None
    for pref in strat:
        for p in (0, 1):
            if discards[p] and discards[p][-1] == pref:
                card = discards[p].pop()
                return card
    return card

def get_from_deck(strat, deck, discards, pick_one=True):
    # Если надо взять две карты и в колоде они есть
    if (not pick_one) and len(deck) >= 2:
        c1 = deck.popleft()
        c2 = deck.popleft()
        i1 = strat.index(c1)
        i2 = strat.index(c2)

        # Выбираем карту с большим приоритетом
        if i1 < i2:
            keep, drop = c1, c2
        else:
            keep, drop = c2, c1
        card = keep

        # Вторую карту в случайный сброс
        pile = random.choice((0,1))
        discards[pile].append(drop)
        return card
    
    # Если одна карта, то берем ее
    if deck:
        return deck.popleft()
    return None


def get_card(strat, deck, discards):
    # Ищем три приоритетные карты в сбросе
    card = get_from_discards(strat[:3], discards)
    
    # Если не нашли карту в сбросе, берем из колоды
    if card is None:
        card = get_from_deck(strat, deck, discards, False)
    return card

### Игровой цикл раунда

In [7]:
def play_round(strat1, strat2, stop_at_7=True, first_player=0):
    # Инициализируем и мешаем колоду
    deck = []
    for card, cnt in STARTING_COUNTS.items():
        deck += [card] * cnt
    random.shuffle(deck)
    deck = deque(deck)
    # Формируем сбрсы
    discards = [deque(), deque()]
    # Инициализируем руки и сыграные пары
    hands = [[], []]
    played_pairs = [0, 0]
    player = first_player
    turns = 0

    while True:
        turns += 1
        # Выбираем стратегию игрока
        strat = strat1 if player == 0 else strat2
        hand = hands[player]
        opp = 1 - player
        opp_hand = hands[opp]

        # Берем карту
        card = get_card(strat, deck, discards)
        if card is None:
            break
        hand.append(card)

        # Играем все возможные пары
        extra_turn = False
        made_pair = True
        while made_pair:
            made_pair = False
            cnt = Counter(hand)

            # Пары рыб, кораблей, крабов
            for p_card in ('fish', 'ship', 'crab'):
                if cnt[p_card] >= 2:
                    # Берем две одинакове карты
                    hand.remove(p_card)
                    hand.remove(p_card)
                    played_pairs[player] += 1
                    if p_card == 'fish':
                        if deck:
                            hand.append(deck.popleft())
                    elif p_card == 'ship':
                        extra_turn = True
                    elif p_card == 'crab':
                        card = get_from_discards(strat, discards)
                        if card:
                            hand.append(card)
                    made_pair = True
                    break
            if made_pair:
                continue

            # Акула и пловец
            if cnt['shark'] >= 1 and cnt['swimmer'] >= 1:
                hand.remove('shark')
                hand.remove('swimmer')
                played_pairs[player] += 1
                # Воруем случайную карты
                if opp_hand:
                    stolen = random.choice(opp_hand)
                    opp_hand.remove(stolen)
                    hand.append(stolen)
                made_pair = True

        # Проверяем условия окончания раунда
        score = played_pairs[player] + score_sets(hand)
        if stop_at_7 and score >= 7:
            break
        if not deck:
            break

        # Определяем, кто ходит
        if extra_turn:
            extra_turn = False
        else:
            player = opp

    # Итоговый счет
    scores = [played_pairs[i] + score_sets(hands[i]) for i in (0,1)]
    return scores, turns

## Симуляция игр

In [8]:
# Симуляция одной игры
def simulate_game(strat1, strat2, stop_at_7=True):
    # Раунд 1: Начинает первый игрок
    s1, t1 = play_round(strat1, strat2, stop_at_7, first_player=0)
    # Раунд 2: начинает второй игрок
    s2, t2 = play_round(strat1, strat2, stop_at_7, first_player=1)
    return (s1, s2), (t1, t2)

## Результаты

In [20]:
def get_results(n_games=1000):
    base = generate_strategy()
    opponents = {
        'reverse': reverse_strategy(base),
        'swap_paired': swap_paired_strategy(base),
        'swap_unpaired': swap_unpaired_strategy(base)
    }
    result = {}
    for name, strat in opponents.items():
        for rule in 'stop_at_7', 'full_deck':
            stop_at_7 = rule == 'stop_at_7'
            win = 0
            turns = 0
            for _ in range(n_games):
                (s1, s2), (t1, t2) = simulate_game(base, strat, stop_at_7)
                turns += t1 + t2
                if s1 > s2:
                    win += 1
            result[(name, rule)] = {
                'win_rate': win / n_games,
                'avg_turns': turns / n_games,
            }
    return pd.DataFrame(result).T

print("Game simulation results:")
print(get_results())

Game simulation results:
                         win_rate  avg_turns
reverse       stop_at_7     0.571     35.333
              full_deck     0.567     75.683
swap_paired   stop_at_7     0.584     43.340
              full_deck     0.543     53.341
swap_unpaired stop_at_7     0.575     38.351
              full_deck     0.567     58.822


## Сравнение стратегий

In [15]:
def compare_strategies(strat1, strat2, stop_at_7=True, n_games=1000):
    df = []
    for _ in range(n_games):
        (s1,s2), _ = simulate_game(strat1,strat2,stop_at_7)
        if s1 == s2:
            continue
        winner = 1 if s1>s2 else 0
        df.append(winner)
    wins = np.array(df)
    n = len(wins)
    count = np.sum(wins == 1)
    stat, pval = proportions_ztest(count, n, value=0.5)
    return stat, pval, count / n

def run_tests(base, opponents):
    results = {}
    for name, opp in opponents.items():
        for rule in 'stop_at_7', 'full_deck':
            stop = (rule == 'stop_at_7')
            stat, p, winrate = compare_strategies(base, opp, stop_at_7=stop)
            results[(name, rule)] = {'z_stat':stat, 'p_value':p, 'win_rate':winrate}
    return pd.DataFrame(results).T

In [ ]:
if __name__ == '__main__':
    random.seed(42)
    base = generate_strategy()
    opponents = {
        'reverse': reverse_strategy(base),
        'swap_paired': swap_paired_strategy(base),
        'swap_unpaired': swap_unpaired_strategy(base)
    }
    tests = run_tests(base, opponents)
    tests['result'] = tests['p_value'].apply(lambda x: 'Принимаем' if x >= 0.5 else 'Отвергаем')
    print("Hypothesis test results:")
    print(tests)

Hypothesis test results:
                           z_stat       p_value  win_rate     result
reverse       stop_at_7  7.183320  6.803854e-13  0.612371  Отвергаем
              full_deck  5.111666  3.193298e-07  0.580060  Отвергаем
swap_paired   stop_at_7  4.916140  8.826748e-07  0.578077  Отвергаем
              full_deck  2.597678  9.385655e-03  0.541369  Отвергаем
swap_unpaired stop_at_7  5.574779  2.478439e-08  0.588054  Отвергаем
              full_deck  3.323728  8.882293e-04  0.552525  Отвергаем
